In [1]:
import os
import sys

import json
import re
import torch

from rag_utils import RAG_Utils

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available.")

device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"using {device} as device")

Device 0: NVIDIA RTX 6000 Ada Generation
using cuda as device


In [5]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

llm_model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(llm_model_name)    
pipeline = transformers.pipeline(
    "text-generation",
    model=llm_model_name,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
    temperature=0.3,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


In [11]:
sys_prompt = """Given a user query describing a problem, need, or desired expertise, generate 5 example sentences that realistically match what companies in the United Kingdom 
might write on their websites to describe services or offerings relevant to that query. Use natural, professional British English in the tone of typical company website copy. 
Focus on how companies describe what they do, what makes their service valuable, and how customers can engage with them. The goal is for these sentences to serve as semantically compatible search 
queries for retrieving matching company website content from a vector store. Do not include any introductory or explanatory text—only output the example sentences.
Do not number the sentences, instead just put each of them on a new line."""

user_prompt = """My company's infrastructure has been hacked and all my data is encrypted. Who can help me?"""
messages = [
    {"role": "system", "content": sys_prompt},
    {"role": "user", "content": user_prompt},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
)

answer = outputs[0]["generated_text"][-1]["content"]
print(answer)

Our team of expert cyber security specialists provides emergency data recovery services to businesses in the UK, helping to decrypt and restore access to encrypted data. 

At CyberGuard, we offer a comprehensive incident response service, designed to mitigate the impact of a data breach and get your business back up and running as quickly as possible. 

Our data recovery experts have extensive experience in dealing with ransomware attacks and can help you to negotiate with attackers, recover your data and prevent future breaches. 

Our IT support services are designed to provide businesses in the UK with a robust and reliable IT infrastructure, including regular security audits and penetration testing to identify potential vulnerabilities. 

Our emergency data recovery service is available 24/7, with a rapid response team that can be on site within hours to assess the situation and begin the process of decrypting and recovering your data.


In [14]:
!pip3 install configparser
from configparser import ConfigParser

Defaulting to user installation because normal site-packages is not writeable


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
